# Azure AI Agents - Connected Agents デモ

このノートブックでは、Azure AI Agentsの**Connected Agents**機能について学習します。

## Connected Agentsとは？

Connected Agentsは、複数のAIエージェントを連携させて、より複雑なタスクを実行する仕組みです。あるエージェントが別のエージェントの機能を「ツール」として利用できるため、専門性の異なるエージェントを組み合わせた強力なAIシステムを構築できます。

### 主な特徴：
- **モジュラー設計**: 各エージェントが特定の機能に特化
- **再利用性**: 一度作成したエージェントを他のエージェントから呼び出し可能
- **スケーラビリティ**: エージェントのネットワークを段階的に拡張
- **責任分離**: 各エージェントが独立して動作

## このデモの構成

1. **Stock Price Agent**: 株価情報を取得する専門エージェント
2. **Main Agent**: Stock Price Agentを「ツール」として使用するメインエージェント
3. **Connected Agent Tool**: 2つのエージェントを接続するブリッジ

## 1. 必要なパッケージのインストール

Azure AI Agentsを使用するために、専用のPythonパッケージをインストールします。

In [ ]:
# 仮想環境内でパッケージをインストール
import sys
!{sys.executable} -m pip install azure-ai-agents

## 2. 環境変数の読み込み

Azure AI Foundryプロジェクトの接続情報とモデルデプロイメント名を読み込みます。

必要な環境変数：
- `PROJECT_CONNECTION_STRING`: Azure AI Foundryプロジェクトのエンドポイント
- `AGENT_MODEL_DEPLOYMENT_NAME`: 使用するモデルのデプロイメント名（例：gpt-4o）

In [ ]:
from dotenv import load_dotenv

load_dotenv('../concierge-agent/.env')

## 3. Azure AI Agentsクライアントの初期化

`AgentsClient`は、Azure AI Agentsサービスとやり取りするためのメインクライアントです。

### 認証方法
- **DefaultAzureCredential**: Azure CLIログイン、マネージドID、環境変数など複数の認証方法を自動的に試行
- プロダクション環境では、マネージドIDまたはサービスプリンシパルの使用を推奨

In [ ]:
import os
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, AgentThreadCreationOptions, ListSortOrder
from azure.identity import DefaultAzureCredential


agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_CONNECTION_STRING"],
    credential=DefaultAzureCredential(),
)

## 4. 専門エージェント（Stock Price Agent）の作成

株価情報を取得する専門のエージェントを作成します。

### パラメータ解説：
- **model**: 使用するLLMモデル（gpt-4oなど）
- **name**: エージェントの識別名
- **instructions**: エージェントの役割と動作を定義するシステムプロンプト
- **tools**: エージェントが使用できるツール（今回は省略、実際にはAPI呼び出しツールなどを設定）

### 注意点：
現在のサンプルでは、実際の株価取得ツールは実装されていません。実用的なシステムでは、Yahoo FinanceやAlpha Vantageなどの株価APIを呼び出すツールを追加する必要があります。

In [ ]:
stock_price_agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="stock_price_bot",
    instructions="Your job is to get the stock price of a company. If you don't know the realtime stock price, return the last known stock price.",
    #tools=... # tools to help the agent get stock prices
)

## 5. Connected Agent Toolの作成

`ConnectedAgentTool`は、既存のエージェントを別のエージェントの「ツール」として使用するためのブリッジです。

### パラメータ解説：
- **id**: 接続対象エージェントのID
- **name**: ツールとしての名前（他のエージェントから呼び出す際の識別子）
- **description**: ツールの機能説明（LLMがツールを選択する際の判断材料）

### Connected Agentの仕組み：
1. メインエージェントがタスクを受信
2. 株価情報が必要と判断した場合、Connected Agent Toolを呼び出し
3. Stock Price Agentが実際の処理を実行
4. 結果がメインエージェントに返される

In [ ]:
connected_agent = ConnectedAgentTool(
    id=stock_price_agent.id, name="connected_agent", description="Gets the stock price of a company"
)

## 6. メインエージェントの作成と実行（パターン1: process_run）

株価取得エージェントを「ツール」として使用するメインエージェントを作成し、実際にタスクを実行します。

### create_thread_and_process_run()の特徴：
- **同期実行**: 処理が完了するまで待機
- **自動完了**: エージェントの応答が完了した状態で結果を取得
- **シンプル**: 一度の呼び出しでスレッド作成から実行完了まで処理

### 実行フロー：
1. メインエージェント作成
2. ユーザーメッセージ「Microsoftの株価は？」を含むスレッド作成
3. エージェントが自動的にConnected Agent Toolを呼び出し
4. Stock Price Agentが株価情報を取得（模擬）
5. 結果をユーザーに返答

In [ ]:
agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a helpful agent, and use the available tools to get stock prices.",
    tools=connected_agent.definitions,
)

print(f"Created agent, ID: {agent.id}")

thread_run = agents_client.create_thread_and_process_run(
    agent_id=agent.id,
    thread=AgentThreadCreationOptions(
        messages=[
            {
                "role": "user",
                "content": "What is the stock price of Microsoft?",
            }
        ]
    ),
)
print(f"Created thread run, ID: {thread_run.id}")
print(f"Thread run status: {thread_run.status}")

messages = agents_client.messages.list(thread_id=thread_run.thread_id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

## 7. メインエージェントの作成と実行（パターン2: create_thread_and_run）

同じタスクを異なるAPIメソッドで実行します。

### create_thread_and_run()の特徴：
- **非同期開始**: 実行を開始するがすぐに制御を返す
- **ステータス確認**: 実行状況を個別に確認可能
- **細かい制御**: 実行の途中でキャンセルや割り込みが可能

### 使い分け：
- **process_run**: シンプルな一回限りのタスク
- **create_thread_and_run**: 長時間実行や複雑な制御が必要なタスク

### リソース管理：
使用後は`delete_agent()`でエージェントを削除し、リソースを適切に解放します。

In [ ]:
agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a helpful agent, and use the available tools to get stock prices.",
    tools=connected_agent.definitions,
)

print(f"Created agent, ID: {agent.id}")

run = agents_client.create_thread_and_run(
    agent_id=agent.id,
    thread=AgentThreadCreationOptions(
        messages=[
            {
                "role": "user",
                "content": "What is the stock price of Microsoft?",
            }
        ]
    ),
)

print(f"Created thread run, ID: {run.id}")
print(f"Thread run status: {run.status}")

messages = agents_client.messages.list(thread_id=run.thread_id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")


# Clean up resources
agents_client.delete_agent(agent.id)

### AI Foundry SDK から呼び出す

## 8. AI Foundry SDK（AIProjectClient）から呼び出す

`AIProjectClient`は、Azure AI Foundryプロジェクト全体を管理するより高レベルなクライアントです。

### AgentsClient vs AIProjectClient：

| 項目 | AgentsClient | AIProjectClient |
|------|-------------|------------------|
| 範囲 | Agents機能に特化 | AI Foundryプロジェクト全体 |
| 用途 | エージェント専用アプリ | 総合的なAIアプリケーション |
| 機能 | エージェント、スレッド、メッセージ | エージェント、評価、モニタリングなど |
| 推奨 | エージェント単体使用 | エンタープライズ開発 |

### AIProjectClientの利点：
- **統合管理**: プロジェクト内のすべてのAIリソースを一元管理
- **コンテキスト管理**: `with`文を使用したリソースの自動管理
- **拡張性**: 将来的な機能追加に対応
- **エンタープライズ機能**: 詳細なログ、メトリクス、セキュリティ機能

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ConnectedAgentTool

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_CONNECTION_STRING"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

connected_agent = ConnectedAgentTool(
    id=stock_price_agent.id, name="connected_agent", description="Gets the stock price of a company"
)

with project_client:
    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        tools=connected_agent.definitions,  # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="What is the weather in Seattle today?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

## まとめ

### Connected Agentsの主要概念

1. **専門化**: 各エージェントが特定の機能に特化
2. **連携**: ConnectedAgentToolを通じたエージェント間の連携
3. **再利用**: 一度作成したエージェントの他の場面での活用
4. **スケーラビリティ**: エージェントネットワークの段階的拡張

### 実装のベストプラクティス

#### 1. エージェント設計
```python
# 明確な役割定義
instructions = "あなたは株価情報取得の専門家です。リアルタイム価格が不明な場合は、最後に知られている価格を返してください。"

# 適切なツール設定
tools = [stock_api_tool, market_data_tool]  # 実際のAPI呼び出しツール
```

#### 2. エラーハンドリング
```python
# 実行状況の確認
if run.status == "failed":
    print(f"実行失敗: {run.last_error}")
    # 適切な回復処理
```

#### 3. リソース管理
```python
# 使用後のクリーンアップ
try:
    # エージェント使用
    pass
finally:
    agents_client.delete_agent(agent.id)
```

### 次のステップ

1. **実際のツール実装**: 株価API、天気API等の実際のデータソースとの統合
2. **エラーハンドリング強化**: ネットワークエラー、API制限等への対応
3. **パフォーマンス最適化**: 並列実行、キャッシュ機能の実装
4. **セキュリティ強化**: 認証、認可、データ暗号化の実装
5. **モニタリング**: ログ、メトリクス、アラートの設定

### 関連リソース

- [Azure AI Agents ドキュメント](https://docs.microsoft.com/azure/ai-services/agents/)
- [Azure AI Foundry](https://docs.microsoft.com/azure/ai-studio/)
- [Connected Agents パターン](https://docs.microsoft.com/azure/ai-services/agents/concepts/connected-agents)
- [エージェント開発ベストプラクティス](https://docs.microsoft.com/azure/ai-services/agents/how-to/best-practices)